# Accessing the Database

In [ ]:
import pymysql
import pandas as pd
import numpy as np

#Connect to MySQL database
cnx = pymysql.connect(user='buspark', password='U@5p1r3!',
                              host='masscourtsplus.org',
                              database='wp_courtdocs')

cursor = cnx.cursor()

## Finding attorney stuff

In [ ]:
query = """

SELECT
    case_number,
    case_type,
    case_status,
    post_title AS attorney
FROM
    cdocs_case_meta_index AS cdocs_meta
        JOIN
    wp_terms ON wp_terms.term_id = cdocs_meta.court_term_id
        JOIN
    wp_term_taxonomy ON wp_term_taxonomy.term_id = cdocs_meta.court_term_id
        JOIN
    wp_terms t1 ON t1.term_id = wp_term_taxonomy.parent
		JOIN
    cdocs_party_assignment_index party_aidx ON party_aidx.case_id = cdocs_meta.post_id
    		JOIN
    wp_posts ON wp_posts.id = party_aidx.attorney_id
    		JOIN
    cdocs_case_action_index action_idx ON action_idx.case_id = party_aidx.case_id
WHERE
    case_status like '%Open%'
    	AND
    action_idx.description like '%debt'
LIMIT 10;

"""

%time

cursor.execute(query)
results = cursor.fetchall()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


In [ ]:
df = pd.DataFrame(results)
df = df.rename({0:'Case Number', 1:'Case Type', 2:'Case Status', 3:'Attorney', 4:'Description'}, axis=1)
df.to_csv('temp.csv', index=False)

df

,Case Number,Case Type,Case Status,Attorney
0,1381CV03494,Contract / Business Cases,Open,"Domestico, Charles J"
1,1473CV00774,Real Property,Open,"Shanley, Edward K"
2,1473CV00774,Real Property,Open,"Leggett, Judith A."
3,1473CV00774,Real Property,Open,"Leggett, Judith A."
4,1473CV01218,Torts,Open,"Domestico, Charles J"
5,WO05P2001EP2,Estates and Administration,Reopened,"Russell, Francis J"
6,WO05P2001EP2,Estates and Administration,Reopened,"Shorr, David Irwin"
7,WO05P2001EP2,Estates and Administration,Reopened,"Russell, Francis J"
8,WO05P2001EP2,Estates and Administration,Reopened,"Shorr, David Irwin"
9,0882CV00725,Contract / Business Cases,Open,"Bolio, Jason S."


# Finding others

In [ ]:
query = """

SELECT
    case_number,
    case_type,
    case_status,
    party_idx.party_name
FROM
    cdocs_case_meta_index AS cdocs_meta
        JOIN
    cdocs_case_action_index action_idx ON action_idx.case_id = cdocs_meta.case_number
    		JOIN
    cdocs_party_assignment_index party_aidx ON party_aidx.case_id = action_idx.case_id
        JOIN
    cdocs_party_index party_idx ON party_idx.post_id = party_aidx.party_id
LIMIT 10;

"""

cursor.execute(query)
%time
results = cursor.fetchall()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


In [ ]:
df = pd.DataFrame(results)
df = df.rename({0:'Case Number', 1:'Case Type', 2:'Case Status', 3:'Party Name'}, axis=1)
# df.to_csv('temp.csv', index=False)

df.head(5)

,Case Number,Case Type,Case Status,Party Name
0,126665SC151,Small Claims,Pending,"Naughton, John Anthony"
1,126665SC151,Small Claims,Pending,"Naughton, John Anthony"
2,126665SC151,Small Claims,Pending,"Naughton, John Anthony"
3,126665SC151,Small Claims,Pending,"Naughton, John Anthony"
4,126665SC151,Small Claims,Pending,"Naughton, John Anthony"


In [ ]:
query = """

SELECT
    civica.case_number,
    cdocs_meta.case_type,
    cdocs_meta.case_status,
    civica.court_department,
    civica.case_title,
    civica.judgments,
    action_idx.action,
    action_idx.description
FROM
    cdocs_case_meta_index AS cdocs_meta
        JOIN
    civica_courtdocs.cases_masscourts_org civica ON civica.wp_post_id = cdocs_meta.post_id
        JOIN
    cdocs_case_action_index action_idx ON action_idx.case_id = cdocs_meta.case_number

"""

%time

cursor.execute(query)
results = cursor.fetchall()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [ ]:
df = pd.DataFrame(results)
df = df.rename({
    0:'Case Number', 
    1:'Case Type', 
    2:'Case Status', 
    3:'Department', 
    4:'Title',
    5:'Judgments',
    6:'Action',
    7:'Description'
    }, axis=1)
df.to_csv('dbquery1.csv', index=False)

df

NameError: name 'pd' is not defined

In [ ]:
query = """

SELECT
    civica.case_number,
    cdocs_meta.case_type,
    cdocs_meta.case_status,
    civica.court_department,
    civica.case_title,
    action_idx.action,
    party_idx.party_name,
    action_idx.description,
    civica.judgments,
    civica.initiating_action,
    civica.docket_information
FROM
    cdocs_case_meta_index AS cdocs_meta
        JOIN
    cdocs_case_action_index action_idx ON action_idx.case_id = cdocs_meta.case_number
        JOIN
    cdocs_party_assignment_index party_aidx ON party_aidx.case_id = cdocs_meta.case_number
        JOIN
    cdocs_party_index party_idx ON party_idx.post_id = party_aidx.party_id
        JOIN
    civica_courtdocs.cases_masscourts_org civica ON civica.wp_post_id = cdocs_meta.post_id
WHERE
    cdocs_meta.case_status like '%closed%'
LIMIT 10;

"""

%time

cursor.execute(query)
results = cursor.fetchall()

In [ ]:
df = pd.DataFrame(results)
# df = df.rename({0:'Case Number', 1:'Case Type', 2:'Case Status', 3:'Title', 4:'Judgment'}, axis=1)
# df.to_csv('temp1.csv', index=False)

df.head(5)

In [ ]:
cursor.close()
cnx.close()